In [1]:
import torch
from torchvision import models
import numpy as np
import pandas as pd
import pickle
import os

In [2]:
# Input and output directories

internal_input_dir = './internal_input_features/'
internal_output_dir = './internal_resnet_output_features/'
if os.path.isdir(internal_output_dir)==False:
      os.mkdir(internal_output_dir)
        
        
external_input_dir = './external_input_features/'
external_output_dir = './external_resnet_output_features/'
if os.path.isdir(external_output_dir)==False:
      os.mkdir(external_output_dir)

In [3]:
# DataLoader
def npy_loader(path):
    return torch.from_numpy(np.load(path, allow_pickle=True)).float()

# Model loading
def load_model():
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    for param in model.parameters():
        param.requires_grad = False
    return model.eval()

# Object save and load
def save_object(obj, filename):
    try:
        with open(filename, "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print(f"Error during pickling object (Possibly unsupported): {ex}")
        
def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print(f"Error during unpickling object (Possibly unsupported): {ex}")

In [4]:
# Function to process images
def model_image_process(model, input_dir, output_dir, subjects, sessions=None):
    for sub in subjects:
        for ses in sessions or ['']:
            if ses:
                in_file = '{}{}_{}.npy'.format(input_dir,sub,ses)
                out_file = '{}{}_{}.npy'.format(output_dir,sub,ses)
            else:
                in_file = '{}{}.npy'.format(input_dir,sub)
                out_file = '{}{}.npy'.format(output_dir,sub)
                
            data = npy_loader(in_file)
            inputs = torch.stack([data])
            out = model(inputs).numpy()
            np.save(out_file, out[0])

In [5]:
# Load model
model = load_model()

# Process internal dataset
df_internal = pd.read_excel('internal_LI_indexes.xlsx').sort_values(by=['Subject'])
model_image_process(model, internal_input_dir, internal_output_dir, df_internal['Subject'], ['ses-1', 'ses-2'])

# Process external dataset
df_external = pd.read_excel('external_LI_indexes.xlsx').sort_values(by=['Subject'])
model_image_process(model, external_input_dir, external_output_dir, df_external['Subject'])